<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>MetPy and Soundings: Calculations</h1>
<h3>Unidata AMS 2021 Student Conference</h3>

<div style="clear:both"></div>
</div>

---

<div style="float:right; width:250 px"><img src="../../instructors/images/metpy-soundings-calculations-preview.png" alt="skew t diagram with Lifting Condensation Level marked, and convective available potential energy (CAPE) and convective inhibition (CIN) shaded" style="height: 300px;"></div>


### Focuses
* Compute common sounding parameters 
* Assign units to variable
* Represent the calculations visually

### Objectives
1. [Read in sample data and assign units](#prep)
1. [Compute the Lifting Condensation Level](#lcl)
1. [Calculate the convective available potential energy (CAPE) and convective inhibition (CIN)](#cape_cin)
1. [Visually represent calculated parameters](#plot)
<br>
<br>
<br>
---

## Imports
We will be using sample data from MetPy in this example.
The raw data can be viewed in your browser at [this location](https://raw.githubusercontent.com/Unidata/MetPy/v0.12.2/staticdata/may4_sounding.txt).
This might be similar to data you would have if you collected it yourself.
We will use the `pandas` library to read in and cleanup the raw data.
`metpy.calc` is used to compute the sounding parameters, such as the lifting condensation level (LCL).
Because we are reading in raw text data, we will `use metpy.units` to make sure units are attached to the numerical data stored read in by `pandas`.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from metpy.units import units

---

## Read in sample data and assign units <a name="prep" />

The first few lines of the raw data file look like:

~~~
-----------------------------------------------------------------------------
   PRES   HGHT   TEMP   DWPT   RELH   MIXR   DRCT   SKNT   THTA   THTE   THTV
    hPa     m      C      C      %    g/kg    deg   knot     K      K      K 
-----------------------------------------------------------------------------
 1000.0     -7                                                               
  959.0    345   22.2   19.0     82  14.64    160     18  298.9  341.8  301.5
  931.3    610   20.2   17.5     84  13.66    165     40  299.4  339.5  301.9
  925.0    671   19.8   17.1     84  13.44    165     38  299.6  339.0  302.0
~~~

Let us define a list to hold the column names of the data we are interested in using:

In [ ]:
col_names = ['pressure', 'height', 'temperature', 'dewpoint', 'direction', 'speed']

Next, we use `pandas` to read in the data contained in the text file, mapping the column name from above to columns `0, 1, 2, 3, 6, and 7`:

In [ ]:
df = pd.read_fwf(get_test_data('may4_sounding.txt', as_file_obj=False),
                 skiprows=5, usecols=[0, 1, 2, 3, 6, 7], names=col_names)

Now that we've read the data into a `DataFrame`, we will use `pandas` to clean up the raw data, dropping any rows with all `NaN` values for `T`, `Td`, and `winds`.

In [ ]:
df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed'), how='all').reset_index(drop=True)

Finally, we use the `metpy.units` package to assign unit to our data:

In [ ]:
p = df['pressure'].values * units.hPa
T = df['temperature'].values * units.degC
Td = df['dewpoint'].values * units.degC
wind_speed = df['speed'].values * units.knots
wind_dir = df['direction'].values * units.degrees

<a href="#top">Top</a>

---

### Compute the Lifting Condensation Level (LCL) <a name="lcl" />

We can use the `lcl` function from `metpy.calc` to compute the LCL.
The MetPy developers have went to great lengths to ensure the calculations available in the package are well documented and tested.
To view this information, you may look in the online [MetPy API documentation](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html), or use `?` to bring up the documentation in the notebook environment, like this:

In [ ]:
mpcalc.lcl?

Because `p`'s first value is ~1000 mb and its last value is ~250 mb, the `0` index is selected for `p`, `T`, and `Td` to lift the parcel from the surface.

<div class="alert alert-block alert-info">
  <b>Note: Sometimes sounding data start at the lowest pressure</b>: If `p` was inverted (i.e. start from low value, 250 mb, to a high value, 1000 mb) the `-1` index should be selected.
</div>

In [ ]:
lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], T[0], Td[0])

<a href="#top">Top</a>

---

## Calculate the convective available potential energy (CAPE) and convective inhibition (CIN) <a name="cape_cin" />

We can also calculate the convective available potential energy (CAPE) and convective inhibition (CIN), this time using the `metpy.calc` function `cape_cin`.
First, let us examine the documentation:

In [ ]:
mpcalc.cape_cin?

While this docstring as viewed in the Jupyter environment is helpful, it is easier to read [in the browser](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.cape_cin.html) (the LaTeX equations are rendered as equations, for example).

As you can see in the docstring, the `cape_cin` function requires a `parcel_profile`, which is the temperature profile of a parcel.
No worries, as MetPy can compute this for us using the `parcel_profile` function:

In [ ]:
prof = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')

Now we can compute the CAPE and CIN values with one function call:

In [ ]:
cape, cin = mpcalc.cape_cin(p, T, Td, prof)
print("CAPE: {:4.1f}".format(cape))
print("CIN: {:4.1f}".format(cin))

<a href="#top">Top</a>

---

## Visually represent calculated parameters <a name="plot" />


The last step will be to visualize our computed parameters.
Building upon the [MetPy SkewT](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/visualization/MetPy-SkewT.ipynb) notebook, we can create a SkewT diagram using matplotlib.
See the comments within the code block below for an explanation about how each element is added to the diagram:

In [ ]:
#####################
# Basic SkewT setup #
#####################

fig = plt.figure(figsize=(9, 9))
add_metpy_logo(fig, 115, 100)
skew = SkewT(fig, rotation=45)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot.
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')
# compute u and v from windspeed and wind direction
u, v = mpcalc.wind_components(wind_speed, wind_dir)
skew.plot_barbs(p, u, v)
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-40, 60)

# Add the relevant special lines
skew.plot_dry_adiabats()
skew.plot_moist_adiabats()
skew.plot_mixing_lines()

#####################################
# Visualize the computed parameters #
#####################################

# Plot the location of the LCL as black dot
skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')

# Plot the profile parcel as black line
skew.plot(p, prof, 'k', linewidth=2)

# Shade areas of CAPE and CIN
skew.shade_cin(p, T, prof)
skew.shade_cape(p, T, prof)

Congratulations!
You have now used python to do some fairly complex calculations for Skew-T analysis based on raw data from a text file.

<a href="#top">Top</a>

---

## Documentation

* [metpy.plots.SkewT](https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.SkewT.html#metpy.plots.SkewT)
* [metpy.calc](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html)

## Other Notebooks

* [MetPy SkewT](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/visualization/MetPy-SkewT.ipynb)

<a href="#top">Top</a>

---